# Azureとlangchainを使ってRAGデモを構築してみる

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain.chat_models import init_chat_model

In [2]:
from langchain.chat_models import init_chat_model

llm = init_chat_model(
    model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
    model_provider="azure_openai",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

In [3]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    model=os.getenv("AZURE_OPENAI_EMBEDDINGS_MODEL_NAME"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

In [4]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [5]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=("https://www.creationline.com/tech-blog/hr/76913",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("content_post", "tech_header")
        )
    ),
)

docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    separators=["\n\n", "\n", "。"],
    chunk_size=100, chunk_overlap=50
)

all_splits = text_splitter.split_documents(docs)

In [7]:
print(all_splits[0])

page_content='2025.03.25
みんなで育むスキルアップ支援制度・正式導入を決定しました

CL HRn-ogasawara92' metadata={'source': 'https://www.creationline.com/tech-blog/hr/76913'}


In [9]:
#　RAG用のプロンプトの読み込み
from langchain_core.prompts.loading import load_prompt

prompt = load_prompt("rlm/rag-prompt")

ValueError: Got unsupported file type 